# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from emoji import UNICODE_EMOJI

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [4]:
import os

filename = 'rtx 3080.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo rtx 3080.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [5]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,@rtx_3080 @thelockalarm don’t let em get frost...,I
1,answering your rtx 3080 questions! - probing p...,R
2,are you still doing my mom https://t.co/yn0umv...,I
3,i guess us customs wins this round. a partner ...,P
4,gonna order when listing on amazon us!\n \nhtt...,I


In [6]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste
0,@kaylasweetx i just want a rtx 3080
1,plan on upgrading with @nvidia's rtx 3080? be ...
2,@insomniacgames i thought the playstation 5 co...
3,@asus_rog @asus_rog are you guys releasing the...
4,@vixlyt @eryxies @charlieintel its 2020. let's...


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [10]:
def separate_emoji(text):
    return ''.join(' ' + char if char in UNICODE_EMOJI else char for char in text).strip()

In [12]:

chars = [",", "'", '"', ".", "!", "?", ";", ":", "[", "]","{", "}", "(", ")", "#", "/",  "\\", "https","+", "@", "*", "-"] 
word_count = dict()
for i, tweet in train.iterrows(): #iterando a base de dados
    piu = tweet.Treinamento #carregando apenas os tweets 
    
    for c in chars: #iterando a listad e cacarteres p/ remover
        piu = piu.replace(c, " ") #trocando os caracteres por " "
        
    for word in separate_emoji(piu).split(): #splitando os tweets em palavras 
        word_count[word] = word_count.get(word, {'I':0, 'P':0, 'R':0}) #garantindo que  a palavra existe no dicionario, caso nao exista ela eh criada
        word_count[word][tweet.Classificação.upper()]+=1 #incrementando o contador do valor correto
        
I = {word for word in word_count if word_count[word]['I'] > 0} #I contem todas as palavras Irrelevantes, seu len() == total de palavras Irrelevantes
P = {word for word in word_count if word_count[word]['P'] > 0} #P contem todas as palavras Pouco relevantes, seu len() == total de palavras Pouco relevantes
R = {word for word in word_count if word_count[word]['R'] > 0} #R contem todas as palavras Relevantes, seu len() == total de palavras Relevantes
print(len(R))
print(word_count)

1308
{'rtx_3080': {'I': 35, 'P': 0, 'R': 0}, 'thelockalarm': {'I': 3, 'P': 0, 'R': 0}, 'don’t': {'I': 2, 'P': 4, 'R': 0}, 'let': {'I': 1, 'P': 6, 'R': 3}, 'em': {'I': 1, 'P': 0, 'R': 1}, 'get': {'I': 7, 'P': 31, 'R': 14}, 'frostbite': {'I': 1, 'P': 0, 'R': 0}, 'answering': {'I': 0, 'P': 0, 'R': 1}, 'your': {'I': 3, 'P': 9, 'R': 13}, 'rtx': {'I': 31, 'P': 201, 'R': 204}, '3080': {'I': 31, 'P': 190, 'R': 170}, 'questions': {'I': 0, 'P': 0, 'R': 1}, 'probing': {'I': 0, 'P': 0, 'R': 1}, 'paul': {'I': 0, 'P': 0, 'R': 1}, '52': {'I': 0, 'P': 0, 'R': 1}, 't': {'I': 27, 'P': 87, 'R': 121}, 'co': {'I': 26, 'P': 72, 'R': 113}, 'e79yohfs6p': {'I': 0, 'P': 0, 'R': 1}, 'via': {'I': 0, 'P': 1, 'R': 9}, 'youtube': {'I': 1, 'P': 2, 'R': 10}, 'are': {'I': 5, 'P': 10, 'R': 22}, 'you': {'I': 13, 'P': 22, 'R': 44}, 'still': {'I': 3, 'P': 4, 'R': 8}, 'doing': {'I': 1, 'P': 0, 'R': 0}, 'my': {'I': 12, 'P': 45, 'R': 17}, 'mom': {'I': 1, 'P': 0, 'R': 0}, 'yn0umvlda5': {'I': 1, 'P': 0, 'R': 0}, 'i': {'I': 31, 

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**